# Lab-05 Logistic Regression

In [1]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

## Training Data

In [3]:
x_data = [[1,2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

## Computing Hypothesis

In [5]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [6]:
W = torch.randn((x_train.size(1), 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [7]:
activation = x_train.matmul(W) + b
output = sigmoid(activation)
output

tensor([[0.7677],
        [0.8932],
        [0.9047],
        [0.9691],
        [0.9838],
        [0.9890]], grad_fn=<MulBackward0>)

#### ※ torch.sigmoid()

In [8]:
activation = x_train.matmul(W) + b
output = torch.sigmoid(activation)
output

tensor([[0.7677],
        [0.8932],
        [0.9047],
        [0.9691],
        [0.9838],
        [0.9890]], grad_fn=<SigmoidBackward>)

## Cost Function

In [9]:
F.binary_cross_entropy(output, y_train)

tensor(1.0176, grad_fn=<BinaryCrossEntropyBackward>)

## Whole Training Procedure

In [10]:
W = torch.randn((x_train.size(1), 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1)

n_epochs = 1000
for epoch in range(n_epochs):
    activation = x_train.matmul(W) + b
    output = torch.sigmoid(activation)
    loss = F.binary_cross_entropy(output, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, loss.item()
        ))

Epoch    0/1000 Cost: 0.873276
Epoch  100/1000 Cost: 0.132809
Epoch  200/1000 Cost: 0.079971
Epoch  300/1000 Cost: 0.057559
Epoch  400/1000 Cost: 0.045093
Epoch  500/1000 Cost: 0.037122
Epoch  600/1000 Cost: 0.031572
Epoch  700/1000 Cost: 0.027480
Epoch  800/1000 Cost: 0.024335
Epoch  900/1000 Cost: 0.021841


In [11]:
for i in range(output.size(0)):
    print(output[i])

tensor([0.0003], grad_fn=<SelectBackward>)
tensor([0.0316], grad_fn=<SelectBackward>)
tensor([0.0390], grad_fn=<SelectBackward>)
tensor([0.9562], grad_fn=<SelectBackward>)
tensor([0.9982], grad_fn=<SelectBackward>)
tensor([0.9997], grad_fn=<SelectBackward>)


## Evaluation

In [12]:
y_pred = output >= torch.FloatTensor([0.5])
y_pred.float()

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])

In [13]:
correct_pred = y_pred.float() == y_train
correct_pred.float()

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [14]:
accuracy = correct_pred.float().mean()
accuracy.item()

1.0

## Higher Implementation with Class

In [15]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [16]:
model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr=1)

In [17]:
n_epochs = 100
for epoch in range(n_epochs):
    output = model(x_train)
    loss = F.binary_cross_entropy(output, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, loss.item()
        ))

Epoch    0/100 Cost: 0.651653
Epoch   10/100 Cost: 1.071482
Epoch   20/100 Cost: 0.603843
Epoch   30/100 Cost: 0.505080
Epoch   40/100 Cost: 0.418513
Epoch   50/100 Cost: 0.336025
Epoch   60/100 Cost: 0.260061
Epoch   70/100 Cost: 0.198890
Epoch   80/100 Cost: 0.162883
Epoch   90/100 Cost: 0.146949


In [18]:
output

tensor([[0.0255],
        [0.1489],
        [0.2822],
        [0.7911],
        [0.9463],
        [0.9828]], grad_fn=<SigmoidBackward>)

In [19]:
y_pred = output >= torch.FloatTensor([0.5])
y_pred.float()

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])